## Load library ##

In [4]:
import pymc3 as pm3
import numpy as np
import numdifftools as ndt
import pandas as pd
from scipy.stats import norm
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
from scipy.optimize import minimize

In [6]:
N = 10000
x = 10 + 2*np.random.randn(N)
y = 5 + x + np.random.randn(N)
df = pd.DataFrame({'y': y, 'x': x})
df['constant'] = 1

print(df.head())
print(df.shape)


           x          y  constant
0   6.745630  12.010155         1
1  11.622892  15.775725         1
2   9.804257  14.261758         1
3  11.773821  16.071597         1
4   9.184171  13.430121         1
(10000, 3)


## OLS Estimation ##

In [13]:
results = sm.OLS(df.y,df[['constant','x']]).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.802
Model:                            OLS   Adj. R-squared:                  0.802
Method:                 Least Squares   F-statistic:                 4.049e+04
Date:                Sat, 11 Nov 2017   Prob (F-statistic):               0.00
Time:                        19:36:03   Log-Likelihood:                -14215.
No. Observations:               10000   AIC:                         2.843e+04
Df Residuals:                    9998   BIC:                         2.845e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
constant       4.9425      0.051     96.993      0.000       4.843       5.042
x              1.0066      0.005    201.228      0.000       0.997       1.016
==============================================================================
Omnibus:                        0.943   Durbin-Watson:                   2.012
Prob(Omnibus):                  0.624   Jarque-Bera (JB):                0.972
Skew:                          -0.020   Prob(JB):                        0.615
Kurtosis:                       2.973   Cond. No.                         52.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [14]:
results.params

constant    4.942471
x           1.006578
dtype: float64

In [10]:
def neg_loglike(theta):
    mu = theta[0] + theta[1] * x
    return -1*norm(mu, theta[2]).logpdf(y).sum()

In [11]:
theta_start = np.array([1, 1, 1])
res = minimize(neg_loglike, theta_start, method = 'Nelder-Mead',
              options = {'disp': True})

Optimization terminated successfully.
         Current function value: 14215.204960
         Iterations: 92
         Function evaluations: 162


In [12]:
hessian_func = ndt.Hessian(neg_loglike, full_output=True)
hessian_ndt, info = hessian_func(res['x'])
se = np.sqrt(np.diag(np.linalg.inv(hessian_ndt)))
results = pd.DataFrame({'parameters': res['x'], 'std err': se})
results.index=['Intercept', 'Slope', 'Sigma']
results.head()

,parameters,std err
Intercept,4.942526,0.050952
Slope,1.006574,0.005002
Sigma,1.002580,0.007089
